In [1]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from typing import Dict, List
import mylib

In [2]:
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)

In [3]:
train = pd.read_csv("input/train.csv", engine="c", low_memory=False)
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19661 entries, 0 to 19660
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Id             19661 non-null  object
 1   pub_title      19661 non-null  object
 2   dataset_title  19661 non-null  object
 3   dataset_label  19661 non-null  object
 4   cleaned_label  19661 non-null  object
dtypes: object(5)
memory usage: 768.1+ KB


In [4]:
gt_map: Dict[str, List[str]] = {}
sep = "|"
for t in tqdm(train.itertuples()):
    pid = getattr(t, "Id")
    cl = getattr(t, "cleaned_label")
    if pid not in gt_map:
        gt_map[pid] = list()
    gt_map[pid].append(mylib.clean_text(cl))

19661it [00:00, 206363.28it/s]


In [5]:
tmp = []
for k, v in tqdm(gt_map.items()):
    is_multi = 0
    if len(v) > 1:
        is_multi = 1
    vs = "|".join(sorted(v))
    tmp.append((k, vs, is_multi))

100%|██████████| 14316/14316 [00:00<00:00, 554259.07it/s]


In [6]:
df = pd.DataFrame(tmp, columns=["Id", "ground_truth", "is_multi"])
df["is_multi"] = df["is_multi"].astype(np.int8) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14316 entries, 0 to 14315
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Id            14316 non-null  object
 1   ground_truth  14316 non-null  object
 2   is_multi      14316 non-null  int8  
dtypes: int8(1), object(2)
memory usage: 237.8+ KB


In [7]:
df.sample(20).head(20)

,Id,ground_truth,is_multi
285,0b80a4a7-8e8b-4f36-8f3d-a6df0e8ec3c8,education longitudinal study|national education longitudinal study,1
9996,b9503746-cda7-4d83-9830-b84e8d3c9f79,slosh model,0
8934,aafd5ee7-69a3-4901-b542-c882a2260721,early childhood longitudinal study,0
13286,9b52509b-8403-4a7f-b55f-8f496dc2dc47,covid 19 open research data,0
3302,918761e1-a549-4e4d-8438-21cd0cdd6e5d,adni|alzheimer s disease neuroimaging initiative adni,1
7402,bef5b889-616b-4d8d-ae6c-a8e57e172d42,trends in international mathematics and science study,0
4195,61f863e5-f545-4252-909c-e1db54670d16,adni|alzheimer s disease neuroimaging initiative adni,1
9216,b9897e1c-dee0-4b66-b46d-2c6c1c097a1e,baccalaureate and beyond|baccalaureate and beyond longitudinal study,1
9395,5e5ee545-b01e-4a19-9a4b-d2617c50b360,national assessment of education progress,0
3562,45a88d54-a216-48f5-a17b-020b372a3a47,adni|alzheimer s disease neuroimaging initiative adni,1


In [8]:
df["is_multi"].value_counts(normalize=True)

0    0.667994
1    0.332006
Name: is_multi, dtype: float64

In [9]:
df.to_parquet("output/train.parquet", index=False)